In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Flatten
# import scipy.ndimage as ndimage

import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
print("Number of GPU available: ",len(tf.config.experimental.list_physical_devices("GPU")))
tf.__version__

Number of GPU available:  0


'2.1.0'

In [10]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]])
      plt.axis('off')

In [11]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return tf.reshape(tf.where(parts[-2] == CLASS_NAMES),[])

In [12]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Cast and normalize the image to dtype float32 in [0,1]
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [13]:
# below functions rotate image using non-tensorflow ndimage py_function
# def random_rotate_image(image):
#   image = ndimage.rotate(image, np.random.uniform(-30, 30), reshape=False)
#   return image
#
# def tf_random_rotate_image(image, label):
#   im_shape = image.shape
#   [image,] = tf.py_function(random_rotate_image, [image], [tf.float32])
#   image.set_shape(im_shape)
#   return image, label
#
# rot_ds = train_ds.map(tf_random_rotate_image)

In [14]:
# Natively, you can only use tf.image to augment.
# You can use tf.py_function to use outside functions such as scipy.ndimage
def augment(image):
  # image = tf.image.convert_image_dtype(image, tf.float32)
  # image = tf.image.resize_with_crop_or_pad(image, IMG_WIDTH+6, IMG_HEIGHT+6) # Add 6 pixels of padding
  # image = tf.image.random_crop(image, size=[IMG_WIDTH, IMG_HEIGHT, 3]) # Random crop back to 28x28
  # image = tf.image.random_contrast(image, lower, upper, seed=None)
  image = tf.image.random_brightness(image, max_delta=0.1) # Random brightness
  image = tf.image.random_flip_left_right(image,seed=5)
  image = tf.image.random_flip_up_down(image, seed=5)
  image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
  # image = tf.image.random_jpeg_quality(image, min_jpeg_quality, max_jpeg_quality, seed=5)
  # image = tf.image.random_saturation(image, lower, upper, seed=5)
  return image


In [15]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label


In [16]:
data_dir=r'C:\Users\kuki\Desktop\Skin\TensorData\train'
# object-oriented pure path
data_dir = pathlib.Path(data_dir)
# list all jpeg images
train_image_count = len(list(data_dir.glob('*\*.jpg')))
train_image_count

0

In [17]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt" and item.name != ".DS_store"])
CLASS_NAMES

array([], dtype=float64)

In [18]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
list_ds

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: C:\\Users\\kuki\\Desktop\\Skin\\TensorData\\train/*/*'

In [ ]:
for f in list_ds.take(5):
  print(f.numpy())

In [ ]:
IMG_HEIGHT = 100
IMG_WIDTH = 100
BATCH_SIZE = 64
epochs = 100

In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", CLASS_NAMES[label.numpy()])

In [ ]:
shuffle_buffer_size = 100 # affect randomness of dataset. it will take first 100 from dataset and shuffle and pick one.
train_ds = (labeled_ds
            .cache("./fibro_train.tfcache")
            .shuffle(buffer_size=shuffle_buffer_size)
            .repeat()
            .map(augment)
            .batch(BATCH_SIZE)
            .prefetch(buffer_size=AUTOTUNE) #time it takes to produce next element
            )

In [ ]:
image_batch, label_batch = next(iter(train_ds))
show_batch(image_batch.numpy(), label_batch.numpy())

In [ ]:
test_data_dir=r'C:\Users\kuki\Desktop\Skin\TensorData\test'
test_data_dir = pathlib.Path(test_data_dir)
test_image_count = len(list(test_data_dir.glob('*\*.jpg')))
test_list_ds = tf.data.Dataset.list_files(str(test_data_dir/'*/*'))
test_labeled_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = (test_labeled_ds
            .cache("./fibro_test.tfcache")
            .shuffle(buffer_size=shuffle_buffer_size)
            .repeat()
            .batch(BATCH_SIZE)
            .prefetch(buffer_size=AUTOTUNE) #time it takes to produce next element
            )
test_image_batch, test_label_batch = next(iter(test_ds))
show_batch(test_image_batch.numpy(), test_label_batch.numpy())

In [ ]:
# val_ds = (
#     test_ds
#     .map(convert, num_parallel_calls=AUTOTUNE)
#     .batch(2*BATCH_SIZE)
# )
# val_image_count =

In [ ]:
model_aug = tf.keras.Sequential([
  Flatten(input_shape=(100,100,3)),
  Dense(128, activation='relu'),
  Dense(2)
])

In [ ]:
model_yesterday= Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2)
])

In [ ]:
model_cnn = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [ ]:
model_new = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  #for multiple choice question

In [ ]:
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=None, cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
# with mirrored_strategy.scope():
#     model = tf.keras.Sequential([
#     Flatten(input_shape=(100,100,3)),
#     Dense(128, activation='relu'),
#     Dense(2)
#     ])
#     model.compile(optimizer='adam',
#               loss=loss_fn,
#               metrics=['accuracy'])

In [ ]:
model = model_aug
model.compile(optimizer='adam',
          loss=loss_fn,
          metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,
      steps_per_epoch=train_image_count // BATCH_SIZE,
      epochs=epochs,
      verbose=2,
      validation_data=test_ds,
      validation_steps=test_image_count // BATCH_SIZE)

In [ ]:
loss_fn2 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) #for A or B question

model2 = model_yesterday
model2.summary()
model2.compile(optimizer='adam',
          loss=loss_fn,
          metrics=['accuracy'])

history2 = model2.fit(train_ds, 
          steps_per_epoch=train_image_count // BATCH_SIZE,
          epochs=epochs,
          verbose=2,
          validation_data=test_ds,
          validation_steps=test_image_count // BATCH_SIZE)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']

loss=history2.history['loss']
val_loss=history2.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(image_batch)
predictions